In [1]:
import sys
sys.path.append("../..")
from src.utils.text_utils import window_slide_split
from datasets import concatenate_datasets, load_dataset

# Prepare QA set

Create a new index dataset that ready for pushing to Vector DB

## Initial 
*Please* ignore if you has runned it before.

In [5]:
import pandas as pd
PATH = "~/Downloads/QA.json"

df = pd.read_json(PATH)
print(df.shape)
df.head()

(170, 2)


,question,answer
0,Thủ tục và cách thức gia hạn đóng học phí như ...,Trong thời gian đóng học phí theo thông báo củ...
1,Xin cho em hỏi về thời gian và cách thức đổi B...,Thời gian đổi biên lai học phí của từng học k...
2,Lệ phí thi chứng chỉ MOS là bao nhiêu,"Trong mỗi đợt thi MOS, Phòng Đại học có thông ..."
3,Tiền tạm thu khi nhập học sẽ được sử dụng như ...,"Đầu năm khi làm thủ tục nhập học, sinh viên s..."
4,Nếu học ngành 2 thì học phí sẽ được tính như t...,"Nếu sinh viên học ngành 2, mức học phí sẽ đượ..."


In [6]:
df['url'] = "https://tuvanhocduong.tdtu.edu.vn/News"

In [7]:
groups = []
groups.extend(["Câu hỏi liên quan học phí"]*9)
groups.extend(["Hoạt động sinh viên"]*2)
groups.extend(["Câu hỏi liên quan Tiếng Anh"]*12)
groups.extend(["Hỗ trợ, khen thưởng, xử lý vi phạm sinh viên"]*18)
groups.extend(["Các câu hỏi khác"]*23)
groups.extend(["Xét và công nhận tốt nghiệp"]*12)
groups.extend(["Đánh giá môn học"]*37)
groups.extend(["Đăng ký môn học"]*33)
groups.extend(["Tổ chức đào tạo"]*8)
groups.extend(["Chương trình đào tạo"]*16)

In [8]:
df['group'] = groups

In [13]:
df

,question,answer,url,group
0,Thủ tục và cách thức gia hạn đóng học phí như ...,Trong thời gian đóng học phí theo thông báo củ...,https://tuvanhocduong.tdtu.edu.vn/News,Câu hỏi liên quan học phí
1,Xin cho em hỏi về thời gian và cách thức đổi B...,Thời gian đổi biên lai học phí của từng học k...,https://tuvanhocduong.tdtu.edu.vn/News,Câu hỏi liên quan học phí
2,Lệ phí thi chứng chỉ MOS là bao nhiêu,"Trong mỗi đợt thi MOS, Phòng Đại học có thông ...",https://tuvanhocduong.tdtu.edu.vn/News,Câu hỏi liên quan học phí
3,Tiền tạm thu khi nhập học sẽ được sử dụng như ...,"Đầu năm khi làm thủ tục nhập học, sinh viên s...",https://tuvanhocduong.tdtu.edu.vn/News,Câu hỏi liên quan học phí
4,Nếu học ngành 2 thì học phí sẽ được tính như t...,"Nếu sinh viên học ngành 2, mức học phí sẽ đượ...",https://tuvanhocduong.tdtu.edu.vn/News,Câu hỏi liên quan học phí
...,...,...,...,...
165,Khi học học phần Tiếng Anh cuối cùng của chươn...,Trong trường hợp sinh viên đủ điều kiện dự thi...,https://tuvanhocduong.tdtu.edu.vn/News,Chương trình đào tạo
166,Sinh viên có vật giá trị trong cặp xách/ Balo ...,Qui định để đồ tại tầng hầm lửng của T...,https://tuvanhocduong.tdtu.edu.vn/News,Chương trình đào tạo
167,Các lỗi vi phạm khi Sinh viên ở ký túc xá có l...,Tùy theo số lần vi phạm và mức độ vi phạm củ...,https://tuvanhocduong.tdtu.edu.vn/News,Chương trình đào tạo
168,Em ở Thành phố Hồ Chí Minh nhưng ở khu vực quậ...,Đối với sinh viên có hộ khẩu thường trú tại T...,https://tuvanhocduong.tdtu.edu.vn/News,Chương trình đào tạo


## Comeback

In [2]:
from huggingface_hub import notebook_login
notebook_login()

## Hugging face

In [3]:
# Replace with your Hub username and repository name
repository_id = "BroDeadlines/QA.FQA_tu_van_hoc_duong"
qa_dataset = load_dataset(repository_id)

In [4]:
filter_qa_dataset = qa_dataset['train'].filter(lambda e: len(e['answer']) > 100)
filter_qa_dataset

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
    num_rows: 144
})

In [5]:
filter_qa_dataset.push_to_hub(repository_id, split="INDEX.medium_index_TDT")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/610 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/QA.FQA_tu_van_hoc_duong/commit/b05c889b450e0b644fd868583b9f95ef2363a74f', commit_message='Upload dataset', commit_description='', oid='b05c889b450e0b644fd868583b9f95ef2363a74f', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
remove_cols = ['question', 'group', 'metadata']
qa_ds = filter_qa_dataset.remove_columns(column_names=remove_cols)
qa_ds = qa_ds.rename_columns({'answer': 'content'})
qa_ds

Dataset({
    features: ['content', 'url', 'doc_id'],
    num_rows: 144
})

# DATASET

In [6]:
from datasets import load_dataset
dataset = load_dataset("BroDeadlines/CRAWL.admission.tdtu.edu.vn_dai-hoc")

In [13]:
remove_cols = ['html_name', 'skipped', 'markdown', 'batch', 'split_markdown', 'slide_markdown']
crawl_ds = dataset['train'].shuffle().select(range(200))
crawl_ds = crawl_ds.remove_columns(remove_cols)
crawl_ds = crawl_ds.rename_columns({'text': 'content'})

In [14]:
crawl_ds

Dataset({
    features: ['url', 'content', 'doc_id'],
    num_rows: 200
})

In [15]:
fin_ds = concatenate_datasets(dsets=[qa_ds, crawl_ds])

fin_ds

Dataset({
    features: ['content', 'url', 'doc_id'],
    num_rows: 344
})

In [16]:
def process_split(row):
    chunks = window_slide_split(row['content'], step=50, chunk_size=1500)
    return {**row,"shards": len(chunks), "splits": chunks}


push_ds = fin_ds.map(process_split)
push_ds

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits'],
    num_rows: 344
})

In [17]:
# from datasets import push_to_hub

# Replace with your Hub username and repository name
repository_id = "BroDeadlines/TEST.basic_test_tdt_dataset"

SPLIT = "INDEX.medium_index_TDT"

push_ds.push_to_hub(repository_id, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/584 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.basic_test_tdt_dataset/commit/72c5239787075d1eafffd23a6e43bc585e689f0a', commit_message='Upload dataset', commit_description='', oid='72c5239787075d1eafffd23a6e43bc585e689f0a', pr_url=None, pr_revision=None, pr_num=None)